In [35]:
%matplotlib qt5

In [36]:
from scipy import ndimage

In [37]:
import pickle
import numpy as np
import matplotlib.pyplot as plt

In [130]:
directory ="/home/ivan/thesis-Ivan/data/"
with open(directory+'S-parameter Coil 1 anticrossings qubit 3 Icoil_3 0.0001 (2).pkl', 'rb') as fhandle:
    data = pickle.load(fhandle)

In [131]:
S21=data[1]['S-parameter'][2]
freqs = data[1]['S-parameter'][1][1]
curr= data[1]['S-parameter'][1][0]

In [132]:
fig, axes = plt.subplots(1,2, figsize=(10,5))
axes[0].pcolormesh(curr, freqs, np.abs(S21.T),cmap='RdBu_r')
axes[1].pcolormesh(curr, freqs, np.angle(S21.T),cmap='RdBu_r')
plt.show()

In [133]:
def anti(S21):
    from scipy import ndimage
    min0=100
    ind=-1
    f = np.angle(S21)
    f_u = np.unwrap(f)
    for i in range(5,50):
        f_u_span = np.median(f_u[:,-1]-f_u[:,0])+2*np.pi*(i/10)
        S21_c = S21*np.exp(-1j*np.linspace(0, f_u_span, len(f[0,:])))
        a = ndimage.sobel(np.angle(S21_c))
        if(min0>np.max(a)):
            min0=np.max(a)
            ind=i    
    print(ind/10)
    f_u_span = np.median(f_u[:,-1]-f_u[:,0])+2*np.pi*(ind/10)
    S21_c = S21[0:]*np.exp(-1j*np.linspace(0, f_u_span, len(f[0,:])))
    plt.figure(1)
    plt.pcolormesh(curr,freqs,np.angle(S21_c).T,cmap='RdBu_r')
    plt.colorbar()
    plt.show()
    plt.figure(2)
    plt.imshow(np.rot90(ndimage.sobel(np.angle(S21_c))))
    plt.colorbar()
    plt.show()
    plt.figure(3)
    plt.pcolormesh(curr,freqs,np.abs(S21).T,cmap='RdBu_r')
    plt.colorbar()
    plt.show()
    return S21_c

def find_where(x,a,curr):
    freq_curr=[]
    for i in range(len(x)):
        if x[i]==a:
            freq_curr.append([curr[i],a])
    return freq_curr

def find_2_fr(z):
    low_fr=z[-1][0]
    high_fr=0
    for i in range(len(z)-1,0,-1):
        if(z[i][0]-low_fr)>1e6:
            high_fr=z[i][0]
            break
    return high_fr

def choose_data (we,curr):
    we_norm=[]
    curr_norm=[]
    for i in range(len(we)-1):
        dif=np.abs(we[i]-we[i+1])/1e5
        if dif<3:
            we_norm.append(we[i])
            curr_norm.append(curr[i])
    return curr_norm,we_norm 

def SortByCount(a):
    return a[1]

def find_smth(S21_c,curr):
    we=[]
    for g in range(len(curr)):
        y=np.diff(np.angle(S21_c[g]))
        we.append(freqs[np.argmax(y)])
    curr,we=choose_data(we,curr)
    z=[[x, we.count(x)] for x in set(we)]
    z.sort()
    z.sort(key=SortByCount)
    line1 = z[-1][0]*np.ones(len(curr))
    line2 = find_2_fr(z)*np.ones(len(curr))
#     plt.plot(curr,we,'ro',linewidth=0.1)
#     plt.plot(curr,line1)
#     plt.plot(curr,line2)
#     plt.show()
    return we,line1[0],line2[0],curr,z

def find_max_min(curr_seq):
    median=[]
    group_freq=[]
    for group in range(len(curr_seq)):
        freq1=curr_seq[group][0][1]
        curr_freq_1=[ curr_seq[group][i][0] for i in range(len(curr_seq[group]))]
        median.append((curr_freq_1[0]+curr_freq_1[-1])/2)
        group_freq.append(freq1)
    return median,group_freq

def split_current(we,fr1,fr2,curr_new):
    curr1,curr2 =find_where(we,fr1,curr_new),find_where(we,fr2,curr_new)
    curr_fr=curr2+curr1
    curr_fr.sort()
    fr_init=curr_fr[0][1]
    ind=0
    curr_seq=[]
    for i in range(len(curr_fr)):
        if curr_fr[i][1]!=fr_init:
            fr_init=curr_fr[i][1]
            curr_seq.append(curr_fr[ind:i])
            ind=i
    if (len(curr_fr)-ind)>1:
        curr_seq.append(curr_fr[ind:len(curr_fr)])
    list_of_remove=[]
    for i in range(len(curr_seq)-1,-1,-1):
        if len(curr_seq[i])<3 :
            list_of_remove.append(i)
    for i in list_of_remove:
        curr_seq.remove(curr_seq[i])
    return curr_seq

1.3


In [140]:
S21_c=anti(S21)
plt.figure(1)
plt.pcolormesh(curr,freqs,np.abs(S21_c).T,cmap='RdBu_r')
plt.colorbar()
plt.show()
plt.figure(2)
plt.pcolormesh(curr,freqs,np.angle(S21_c).T,cmap='RdBu_r')
plt.colorbar()
plt.show()


In [136]:
we,fr1,fr2,curr_new=find_smth(S21_c,curr)[:4]

In [142]:
curr_seq=split_current(we,fr1,fr2,curr_new)
middle_curr,middle_freq=find_max_min(curr_seq)
print(middle_curr,middle_freq)
plt.plot(middle_curr,middle_freq,'go')
plt.show()

[-0.0014599999999999999, -0.00013499999999999992, 0.0014900000000000002] [6808250000.0, 6812450000.0, 6808250000.0]


In [141]:
L_q3_C1*(0.0014900000000000002+0.0014599999999999999)/F0

1.0649819494584838

In [21]:
L_matrix={1:{
            "Coil":[],
            "Qubit":[],
            "Frequency":[],
            "Current":[] 
            },
          2:{
            "Coil":[],
            "Qubit":[],
            "Frequency":[],
            "Current":[] 
            },
          3:{
            "Coil":[],
            "Qubit":[],
            "Frequency":[],
            "Current":[] 
            },
          4:{
            "Coil":[],
            "Qubit":[],
            "Frequency":[],
            "Current":[] 
            }
         }

In [59]:
num=1
L_matrix[num]['Coil'].append(1)
L_matrix[num]['Qubit'].append(1)
L_matrix[num]['Frequency'].append(middle_freq[:2])
L_matrix[num]['Current'].append(middle_curr[:2])

In [67]:
L_matrix[4]["Qubit"][0]=3

In [68]:
L_matrix

{1: {'Coil': [1],
  'Current': [[-0.0012700000000000001, 0.00085000000000000006]],
  'Frequency': [[6532400000.0, 6529100000.0]],
  'Qubit': [1]},
 2: {'Coil': [1],
  'Current': [[-0.00088499999999999994, 0.00050000000000000023]],
  'Frequency': [[6813050000.0, 6808700000.0]],
  'Qubit': [3]},
 3: {'Coil': [3],
  'Current': [[-9.5500000000000017e-05, 3.5499999999999982e-05]],
  'Frequency': [[6529100000.0, 6532400000.0]],
  'Qubit': [1]},
 4: {'Coil': [3],
  'Current': [[-9.3000000000000011e-05, 2.6999999999999979e-05]],
  'Frequency': [[6808400000.0, 6812900000.0]],
  'Qubit': [3]}}

In [31]:
F0=2.0678e-7

In [122]:
I=-0.000165

In [143]:
I2=-0.00013499999999999992

In [17]:
F0_half=2.0678e-7/2
F0_half

1.0339e-07

In [154]:
F_ext_2=(F0_half-(L_q3_C1*I2+L_q3_C3*0.1e-3))/F0
F_ext_2

0.13206979542719605

In [156]:
F_ext_1=(F0_half-(L_q1_C1*I+L_q1_C3*0.1e-3))/F0
F_ext_1

0.15723570502664547

In [161]:
I_3=(F0_half-F_ext_2*F0-L_q3_C1*I_1)/L_q3_C3
I_3

9.35366271500034e-05

In [160]:
I_1=(-F0_half+F_ext_2*F0-(-F0_half+F_ext_1*F0)*L_q3_C3/L_q1_C3)/(L_q3_C3*L_q1_C1/L_q1_C3-L_q3_C1)
I_1

-6.040190502295583e-05

In [149]:
F_ext_1,F_ext_2

(0.15723570502664547, 0.13206979542719605)

In [150]:
I_1=(-F0_half+F_ext_2*F0+(F0_half-F_ext_1*F0)*L_q3_C3/L_q1_C3)/(L_q3_C3/L_q1_C3*L_q1_C1-L_q3_C1)

-6.040190502295583e-05

In [146]:
F_ext_1=(F0_half-(L_q1_C1*I+L_q1_C3*0.1e-3))/F0

In [93]:
(L_q1_C1*(0.00085000000000000006))/F0

0.2004716981132075

In [94]:
((L_q1_C3*(-9.5500000000000017e-05)))/F0

-0.36450381679389326

In [96]:
(L_q3_C1*(0.00050000000000000023))/F0

0.18050541516245494

In [95]:
((L_q3_C3*(-9.3000000000000011e-05)))/F0

-0.38750000000000007

In [80]:
(F0_half-(L_q1_C1*(-0.0012700000000000001)))/F0

0.7995283018867924

In [81]:
(F0_half-(L_q3_C3*2.6999999999999979e-05))/F0

0.38750000000000007

In [82]:
(F0_half-(L_q3_C1*-0.00088499999999999994))/F0

0.8194945848375451

In [83]:
(F0_half-(L_q1_C3*3.5499999999999982e-05))/F0

0.36450381679389315

In [28]:
deltaI_q1_C1=0.00085000000000000006+0.0012700000000000001
L_q1_C1=F0_half/deltaI_q1_C1
L_q1_C1/1e-6

48.768867924528294

In [23]:
deltaI_q3_C1=0.00050000000000000023+0.00088499999999999994
L_q3_C1=F0_half/deltaI_q3_C1
L_q3_C1/1e-6

74.64981949458483

In [25]:
deltaI_q1_C3=9.5500000000000017e-05+3.5499999999999982e-05
L_q1_C3=F0_half/deltaI_q1_C3
L_q1_C3/1e-6

789.2366412213743

In [26]:
deltaI_q3_C3=9.3000000000000011e-05+2.6999999999999979e-05
L_q3_C3=F0_half/deltaI_q3_C3
L_q3_C3/1e-6

861.5833333333335

In [ ]:
# curr1,curr2 =find_where(we,fr1,curr_new),find_where(we,fr2,curr_new)
# curr_fr=curr2+curr1
# curr_fr.sort()
# curr_fr

In [ ]:
curr1,curr2 =find_where(we,fr1,curr_new),find_where(we,fr2,curr_new)
curr1,curr2 

In [41]:
curq=[curr_fr[i][0]for i in range(len(curr_fr))]

In [ ]:
plt.plot()

In [28]:
for j in range(len(curr1)):
    

ValueError: either both or neither of x and y should be given

In [23]:
curr_fr

[-0.00089999999999999998,
 -0.00087999999999999992,
 -0.00085999999999999987,
 -0.00084000000000000003,
 -0.00081999999999999998,
 -0.00079999999999999993,
 -0.00077999999999999988,
 -0.00076000000000000004,
 -0.00073999999999999999,
 2.0000000000000052e-05,
 4.0000000000000105e-05,
 6.0000000000000157e-05,
 8.000000000000021e-05,
 0.00010000000000000026,
 0.00012000000000000031,
 0.00013999999999999993,
 0.00015999999999999999,
 0.00018000000000000004,
 0.00020000000000000009,
 0.00022000000000000014,
 0.0002400000000000002,
 0.00026000000000000025,
 0.0002800000000000003,
 0.00030000000000000035,
 0.00031999999999999997,
 0.00034000000000000002,
 0.00036000000000000008,
 0.0011400000000000004,
 0.00116,
 0.0011800000000000001,
 0.0012000000000000001,
 0.0012200000000000002,
 0.0012400000000000002]

In [24]:
plt.plot(curr2,fr2*np.ones(len(curr2)),'yo')
plt.plot(curr1,fr1*np.ones(len(curr1)),'go')
plt.show()

In [25]:

center_fr1,center_fr2 =(curr2[0]-curr2[-1])/2+curr2[-1],(curr1[0]-curr1[-1])/2+curr1[-1]
print((fr1,center_fr1),(fr2,center_fr2))
plt.plot([center_fr1,center_fr2],[fr2,fr1],'ko')
plt.show()

TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [395]:
line2 = ((fr1-fr2)/2+fr2)*np.ones(len(curr))
plt.plot(curr,line2)

In [387]:
f = np.angle(S21[0:])
f_u = np.unwrap(f)
S21_c = []
# for i 
f_u_span = np.median(f_u[:,-1]-f_u[:,0])+2*np.pi*(0.8)
S21_c = S21[0:]*np.exp(-1j*np.linspace(0, f_u_span, len(f[0,:])))
plt.figure()
plt.pcolormesh(curr,freqs,np.angle(S21_c[0:]).T,cmap='RdBu_r')
plt.colorbar()
plt.show()
# plt.savefig('Anti_211.png',dpi=300)

In [352]:
find_2_fr(zxc)

6811400000.0

In [273]:
line = zxc[-4][0]*np.ones(len(curr))
plt.plot(curr,line)
plt.show()

In [223]:
we=[]
for g in range(len(curr)):
    y=np.diff(np.angle(S21_c[g]))
    we.append(freqs[np.argmax(y)])

In [329]:
for i in range(len(np.diff(we))):
    tr=0
    if np.abs(np.diff(we)[i])/1e6>1:
        tr=1
        we[i+1]=6.809e9
    else:
        tr=0

In [330]:
plt.plot(curr,we,'ro',linewidth=0.1)
plt.show()

In [230]:
xcv=[d[-1:][i][0]*np.ones(len(curr)) for i in range(1)]
xcv[0]

array([  6.66070000e+09,   6.66070000e+09,   6.66070000e+09,
         6.66070000e+09,   6.66070000e+09,   6.66070000e+09,
         6.66070000e+09,   6.66070000e+09,   6.66070000e+09,
         6.66070000e+09,   6.66070000e+09,   6.66070000e+09,
         6.66070000e+09,   6.66070000e+09,   6.66070000e+09,
         6.66070000e+09,   6.66070000e+09,   6.66070000e+09,
         6.66070000e+09,   6.66070000e+09,   6.66070000e+09,
         6.66070000e+09,   6.66070000e+09,   6.66070000e+09,
         6.66070000e+09,   6.66070000e+09,   6.66070000e+09,
         6.66070000e+09,   6.66070000e+09,   6.66070000e+09,
         6.66070000e+09,   6.66070000e+09,   6.66070000e+09,
         6.66070000e+09,   6.66070000e+09,   6.66070000e+09,
         6.66070000e+09,   6.66070000e+09,   6.66070000e+09,
         6.66070000e+09,   6.66070000e+09,   6.66070000e+09,
         6.66070000e+09,   6.66070000e+09,   6.66070000e+09,
         6.66070000e+09,   6.66070000e+09,   6.66070000e+09,
         6.66070000e+09,

In [206]:
(xcv[1]-xcv[0])[0]/2+xcv[0][0]

6531000000.0

In [231]:
for i in range(1):
    plt.plot(curr,xcv[i])
plt.show()

In [233]:
d=z[-1][0]
d

6660700000.0

In [225]:
def SortByCount(a):
    return a[1]
z=[[x, we.count(x)] for x in set(we)]
z.sort(key=SortByCount)
z

[[6668500000.0, 1],
 [6663100000.0, 1],
 [6662500000.0, 1],
 [6661300000.0, 1],
 [6673500000.0, 1],
 [6665300000.0, 1],
 [6664900000.0, 1],
 [6665700000.0, 1],
 [6667300000.0, 1],
 [6669300000.0, 1],
 [6655700000.0, 1],
 [6668100000.0, 1],
 [6674500000.0, 1],
 [6671300000.0, 1],
 [6656500000.0, 1],
 [6657700000.0, 1],
 [6655300000.0, 1],
 [6657900000.0, 1],
 [6670700000.0, 1],
 [6657300000.0, 1],
 [6656100000.0, 1],
 [6654900000.0, 1],
 [6658900000.0, 1],
 [6665500000.0, 1],
 [6665100000.0, 1],
 [6666300000.0, 1],
 [6666700000.0, 1],
 [6671100000.0, 1],
 [6655100000.0, 1],
 [6658700000.0, 1],
 [6672100000.0, 1],
 [6659100000.0, 2],
 [6659900000.0, 2],
 [6659500000.0, 3],
 [6664500000.0, 3],
 [6659700000.0, 3],
 [6658300000.0, 3],
 [6660100000.0, 4],
 [6664700000.0, 6],
 [6660300000.0, 7],
 [6660500000.0, 10],
 [6660700000.0, 27]]

<built-in method items of dict object at 0x7fa82c9908c8>


In [100]:
x=set(we)
x.

AttributeError: 'set' object has no attribute 'shape'

In [122]:
plt.plot(curr,we,'ro')
plt.show()

In [52]:
a = ndimage.sobel(np.abs(S21_c))
plt.imshow(a)
plt.colorbar()
plt.show()

In [517]:
asdf=[1,1,2]
asz=np.where(asdf>1)
asz

TypeError: unorderable types: list() > int()

In [506]:
qaz=[]
for j in range(len(a[0])):
#     qw
    qwe=[int(a[i][j])for i in range(len(a[0]))]
#     plt.plot(range(len(qwe)),qwe,"ro")
#     plt.show()
    qaz

In [582]:
qw=43
qwe=[int(a[i][qw])for i in range(len(a[0]))]
plt.plot(range(len(qwe)),qwe,"ro")
plt.show()

In [519]:
g=30
plt.plot(range(len(a[g])),a[g])
plt.show()

In [438]:
np.max(a)

24.767435841806712

In [559]:
min0=100
ind=-1
for i in range(5,50):
    print(i)
    f_u_span = np.median(f_u[:,-1]-f_u[:,0])+2*np.pi*(i/10)
    S21_c = S21[0:]*np.exp(-1j*np.linspace(0, f_u_span, len(f[0,:])))
    a = ndimage.sobel(np.angle(S21_c))
    if(min0>np.max(a)):
        min0=np.max(a)
        ind=i    
    print(np.max(a))
print(ind/10)

5
1.7540949501
6
1.70382946764
7
1.65356398518
8
1.60329850273
9
12.4069318172
10
24.8430030168
11
24.8240885548
12
24.7743321429
13
24.6115728495
14
24.6504888953
15
24.6314475973
16
24.5480289568
17
24.4443193754
18
24.396308446
19
24.3489599685
20
24.8410250879
21
24.5740075079
22
24.3354856928
23
24.2307293121
24
24.2207587477
25
24.1714900512
26
24.1087665201
27
23.9307196813
28
23.9564434665
29
23.9450581105
30
23.8947926281
31
23.9349768913
32
23.8462536165
33
23.8106161649
34
24.1553326958
35
24.0545067822
36
23.5787490551
37
23.5673074757
38
23.73304853
39
23.4207722545
40
23.5101928206
41
23.5740477242
42
24.2643025677
43
24.6773427587
44
24.6295232037
45
24.4731264308
46
24.2242925688
47
23.4306032901
48
23.4552205777
49
24.0348154728
0.8


In [551]:
ind

8

In [408]:
max(a[g])

24.216214507816172

In [378]:
np.mean(a[0])

0.00088531274598642885

In [349]:
b=a[0][a[0]>20]

In [350]:
b

array([ 23.65009291])

In [157]:
plt.plot(freqs,np.imag(S21_c[int(x[np.argmin(func(x,popt[0],popt[1],popt[2]))])]))
plt.show()

NameError: name 'x' is not defined

In [80]:
amps = 2*abs(np.fft.fft(np.imag(S21_c[80])))/len(np.imag(S21_c[80]))
amps = amps[0:int(len(amps)/2)]
amps = amps[0:50] 
freqs1=np.arange(0, len(amps), 1)

In [85]:
amps[np.argmax(amps)]

0.034328265671564009

In [86]:
plt.plot(freqs1,amps,'ro')

In [27]:
plt.plot(range(len(argmaxs)),freqs[argmaxs])
plt.show()

In [12]:
plt.plot(range(26,62),freqs[argmaxs[26:62]])
plt.show()

In [24]:
plt.plot(range(29,59),freqs[argmaxs[29:59]])
plt.show()

In [285]:
for i in range(0,5):
    plt.plot(freqs[:-1],dif[i])
plt.show()

In [22]:
from scipy.optimize import curve_fit

In [17]:
def func(x,a,b,c):
    return a*x*x+b*x+c

In [50]:
def sin(x,a,b,c,d):
    return a*np.sin(b*x+c)+d

In [53]:
popt=curve_fit(sin,freqs[:50]/1e9,y[:50]*100,maxfev=1000000)[0]
popt

array([ -8.28114246e+03,   2.57372490e+00,  -1.20844873e+01,
        -8.28074392e+03])

In [36]:
popt=curve_fit(func,range(29,59),freqs[argmaxs[29:59]])[0]
popt

NameError: name 'func' is not defined

In [55]:
plt.plot(freqs[:50]/1e9,sin(freqs[:50]/1e9,popt[0],popt[1],popt[2],popt[3]))

In [22]:
min(func(x,popt[0],popt[1],popt[2])),curr[int(x[np.argmin(func(x,popt[0],popt[1],popt[2]))])]

(6532248509.1985798, -0.00096250000000000003)

In [25]:
freqs[int(x[np.argmin(func(x,popt[0],popt[1],popt[2]))])]

6500200000.0

In [20]:
x= np.linspace(29,59,1000)

In [95]:
curr[43]

-0.00096250000000000003

In [97]:
min(func(np.linspace(26,62,1000),popt[0],popt[1],popt[2]))

6532301082.6936073

In [91]:
np.argmin(func(np.linspace(26,62,1000),popt[0],popt[1],popt[2]))

477

In [288]:
fig, axes = plt.subplots(1,2, figsize=(10,5))
axes[0].pcolormesh(curr, freqs, np.log10(np.abs(S21).T),cmap='RdBu_r')
axes[1].pcolormesh(curr,freqs,np.angle(S21_c).T,cmap='RdBu_r')

In [308]:
a.shape

(101, 101)

divide this picture on two pictures

In [293]:
, misc
import matplotlib.pyplot as plt
fig = plt.figure()
# plt.gray()  # show the filtered result in grayscale
ax1 = fig.add_subplot(121)  # left side
ax2 = fig.add_subplot(122)  # right side
ascent = misc.ascent()
result = ndimage.sobel(np.angle(S21_c))
ax1.imshow(ascent)
ax2.imshow(result.T)
plt.show()

## First qubit

In [27]:
cur1,cur2=0,100
fr1,fr2=0,200
curr_1=curr[cur1:cur2]*1e4
freqs_1=freqs[fr1:fr2]/1e9
S21_p_1=np.angle(S21_c[cur1:cur2,fr1:fr2])
S21_a_1=np.abs(S21[cur1:cur2,fr1:fr2])
S21_p_1_1=np.angle(S21[:,fr1:fr2])

In [28]:
plt.pcolormesh(curr_1,freqs_1,np.log10(np.abs(S21_a_1)).T,cmap="RdBu_r")
plt.colorbar()

In [30]:
plt.pcolormesh(curr_1, freqs_1, np.log10(S21_a_1.T),cmap='RdBu_r')
plt.colorbar()
plt.savefig('Anti-222.png')

In [28]:
fig, axes = plt.subplots(1,2, figsize=(10,5))
axes[0].pcolormesh(curr_1, freqs_1, np.log10(S21_a_1.T),cmap='RdBu_r')
#axes[1].pcolormesh(curr_1, freqs_1, np.log10(np.abs(S21_p_1.T))*10,cmap='RdBu_r')

TypeError: 'AxesSubplot' object does not support indexing

## second qubit

In [23]:
curr_2=curr[:250]
freqs_2=freqs[600:1000]
S21_p_2=np.angle(S21_c[:250,600:1000])
S21_a_2=S21[:250,600:1000]

In [22]:
fig, axes = plt.subplots(1,1, figsize=(10,5))
axes[0].pcolormesh(curr_2, freqs_2,np.log10(np.abs(S21_a_2.T)),cmap='RdBu_r')
# axes[1].pcolormesh(curr_2, freqs_2, np.log10(np.abs(S21_p_2).T),cmap='RdBu_r')

TypeError: 'AxesSubplot' object does not support indexing

## Modeling

In [11]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import pi

from qutip import *

In [15]:
#N = 20
Nt = 2 # number of transmon states considered
Nr = 2 # number of resonator mode states considered

# system parameter

wr = 6.532 * 2 * pi   
wq = 8.2* 2 * pi
g = 0.060 * 2* pi

In [16]:
a = tensor(destroy(Nr), qeye(Nt))
nc = a.dag() * a
xc = a + a.dag()
yc = 1j*(a - a.dag())

# atomic operators
sm = tensor(qeye(Nr), destroy(2))
sz = tensor(qeye(Nr), sigmaz())
sx = tensor(qeye(Nr), sigmax())
nq = sm.dag() * sm
xq = sm + sm.dag()

I = tensor(qeye(Nr), qeye(Nt))

In [17]:
H = wr * (a.dag() * a + I/2.0) + (wq / 2.0) * sz + g * xc * xq # Jaynes-Cummings Hamiltonian
#RWA

In [19]:
%%time
flux_points = 1025
spectral_lines = np.zeros((2, flux_points))
wq_flux_dep = np.zeros((flux_points))
fluxes = np.linspace(-1,1, flux_points) #in flux quanta
for flux_id, flux in enumerate(fluxes):
    wq_flux = wq*np.sqrt(np.abs(np.cos(flux*np.pi)))
    H_spectral = wr * (a.dag() * a + I/2.0) + (wq_flux / 2.0) * sz + g * xc * xq # Jaynes-Cummings Hamiltonian
    energies= H_spectral.eigenenergies()
    ground_state_energy = energies[0]
    spectral_lines[:, flux_id] = np.sort(energies[1:3]-ground_state_energy)/(2*np.pi) # 0->1 and 1->2 transitions only

CPU times: user 1.99 s, sys: 4 ms, total: 1.99 s
Wall time: 1.97 s


In [20]:
energies

array([ -5.24171192,  35.78804655,  46.2954863 ,  87.32524477])

In [21]:
plt.plot(fluxes, spectral_lines.T, linewidth=3)
plt.show()

In [9]:
%matplotlib qt5

In [53]:
fl=0.27
fig, axes = plt.subplots(1,2, figsize=(10,5))
axes[0].pcolormesh(curr_1, freqs_1, np.log10(S21_a_1.T),cmap='RdBu_r')
axes[1].pcolormesh(curr_1, freqs_1, np.log10(np.abs(S21_p_1.T))*10,cmap='RdBu_r')
ax_lims = axes[1].get_xlim()
axes[1].set_ylim(axes[0].get_ylim())
axes[1].set_xlim(ax_lims)
a=axes[1].plot(fluxes+fl, spectral_lines.T, linewidth=3)[0]

def f(fl):
#     global fluxes,spectral_lines
#     print(fluxes,spectral_lines.T)
    a.set_data(fluxes, spectral_lines[1].T)
    plt.show()
# axes[1].plot(curr_1,6*np.abs(curr_1))
interactive(f, fl=(0.1,1.0,0.01))

interactive(children=(FloatSlider(value=0.55, description='fl', max=1.0, min=0.1, step=0.01), Output()), _dom_classes=('widget-interact',))

In [71]:
plt.figure(1)
ax1= plt.imshow(np.log10(S21_a_1.T),cmap='RdBu_r')
plt.title("real")
plt.colorbar()

In [65]:
fl=0.27
fig, axes = plt.subplots(1,2, figsize=(10,5))
axes[0].pcolormesh(curr_1, freqs_1, np.log10(S21_a_1.T),cmap='RdBu_r')
axes[1].pcolormesh(curr_1, freqs_1, np.log10(np.abs(S21_p_1.T))*10,cmap='RdBu_r')
ax_lims = axes[1].get_xlim()
axes[1].set_ylim(axes[0].get_ylim())
axes[1].set_xlim(ax_lims)
axes[1].plot(fluxes+fl, spectral_lines.T, linewidth=3)

In [22]:
from ipywidgets import interactive

In [83]:
%matplotlib notebook

In [23]:
plt.figure(1)
ax=plt.pcolormesh(curr_1, freqs_1, np.log10(np.abs(S21_p_1.T))*10,cmap='RdBu_r')
plt.colorbar()
gr=plt.plot([],[])[0]

def f(r,t,fl):
    wr = 6.657 * 2 * pi      # resonator frequency, GHz
    wq = 12.0014 * 2 * pi # qubit frequency (peak value)
    g = 0.070 * 2* pi
    print(r+t/1000)
    H = wr * (a.dag() * a + I/2.0) + (wq / 2.0) * sz + g * xc * xq # Jaynes-Cummings Hamiltonian
    #RWA
    # plotting low-power spectrum (anticrossing positions), assuming that the qubit is an (actually linear) LC oscillator
    # spectral lines. The lower part where the qubit stops being a transmon is wrong (but we don't care about it anyway)
    flux_points = 257
    spectral_lines = np.zeros((2, flux_points))
    wq_flux_dep = np.zeros((flux_points))
    fluxes = np.linspace(curr_1[0]-0.5, curr_1[len(curr_1)-1], flux_points) #in flux quanta
    for flux_id, flux in enumerate(fluxes):
        wq_flux = wq*np.sqrt(np.abs(np.cos(flux*np.pi)))
        H_spectral = wr * (a.dag() * a + I/2.0) + (wq_flux / 2.0) * sz + g * xc * xq # Jaynes-Cummings Hamiltonian
        energies, eigenvectors = H_spectral.eigenstates()
        ground_state_energy = energies[0]
        spectral_lines[:, flux_id] = np.sort(energies[1:3]-ground_state_energy)/(2*np.pi) # 0->1 and 1->2 transitions only
        wq_flux_dep[flux_id] = wq_flux
    gr.set_data(fluxes+fl, spectral_lines.T)
    plt.show()
    
    #plt.plot(fluxes, wq_flux_dep/(2*np.pi))

interactive_plot = interactive(f, fl=(0.1,1.0,0.01),r=(0.01,1.0,0.01),t=(1,10,1))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

NameError: name 'curr_1' is not defined

In [53]:
%matplotlib qt5
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np
plt.figure()
x = np.linspace(-10, 10, num=1000)
plt.figure(1)
plt.pcolormesh(curr_1, freqs_1, np.log10(np.abs(S21_a_1.T))*10,cmap='RdBu_r')
b, =plt.plot([],[])
c, =plt.plot([],[])
plt.ylim(freqs_1[0],freqs_1[len(freqs_1)-1])
plt.xlim(curr_1[0],curr_1[len(curr_1)-1])
plt.show()
fl=-4
def f( fl,q2,q3,a1):
    wr = (6.531) * 2 * pi   
    wq = (8.6+q2) * 2 * pi
    g = (0.070+q3/1000) * 2* pi
    H = wr * (a.dag() * a + I/2.0) + (wq / 2.0) * sz + g * xc * xq 
    flux_points = 257
    spectral_lines = np.zeros((2, flux_points))
    wq_flux_dep = np.zeros((flux_points))
    fluxes = np.linspace(curr_1[0]-0.5, curr_1[len(curr_1)-1], flux_points) #in flux quanta
    for flux_id, flux in enumerate(fluxes):
        wq_flux = wq*np.sqrt(np.abs(np.cos(flux*np.pi/(11+4.2+a1))))
        H_spectral = wr * (a.dag() * a + I/2.0) + (wq_flux / 2.0) * sz + g * xc * xq # Jaynes-Cummings Hamiltonian
        energies, eigenvectors = H_spectral.eigenstates()
        ground_state_energy = energies[0]
        spectral_lines[:, flux_id] = np.sort(energies[1:3]-ground_state_energy)/(2*np.pi) # 0->1 and 1->2 transitions only
        wq_flux_dep[flux_id] = wq_flux
    b.set_data(fluxes+fl, spectral_lines[0])
    c.set_data(fluxes+fl, spectral_lines[1])
#     plt.savefig("anticro_ampl_1.png",dpi=300)
interactive(f,
            a1=(-2, 5, 0.1),
            fl=(-20, 20, 0.1),
            q2=(-3, 3, 0.0001),
            q3=(-10, 10, 0.1))

interactive(children=(FloatSlider(value=0.0, description='fl', max=20.0, min=-20.0), FloatSlider(value=0.0, description='q2', max=3.0, min=-3.0, step=0.0001), FloatSlider(value=0.0, description='q3', max=10.0, min=-10.0), FloatSlider(value=1.0, description='a1', max=5.0, min=-2.0), Output()), _dom_classes=('widget-interact',))

In [41]:
%matplotlib qt5
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np
plt.figure()
x = np.linspace(-10, 10, num=1000)
plt.figure(1)
plt.pcolormesh(curr_1, freqs_1, np.log10(np.abs(S21_a_1.T))*10,cmap='RdBu_r')
b, =plt.plot([],[])
c, =plt.plot([],[])
plt.ylim(freqs_1[0],freqs_1[len(freqs_1)-1])
plt.xlim(curr_1[0],curr_1[len(curr_1)-1])
plt.show()
wr = (6.532) * 2 * pi   
wq = (8.6) * 2 * pi
g = (0.00) * 2* pi
H = wr * (a.dag() * a + I/2.0) + (wq / 2.0) * sz + g * xc * xq 
flux_points = 257
spectral_lines = np.zeros((2, flux_points))
wq_flux_dep = np.zeros((flux_points))
fluxes = np.linspace(curr_1[0]-0.5, curr_1[len(curr_1)-1], flux_points) #in flux quanta
for flux_id, flux in enumerate(fluxes):
        wq_flux = wq*np.sqrt(np.abs(np.cos(flux*np.pi/12)))
        H_spectral = wr * (a.dag() * a + I/2.0) + (wq_flux / 2.0) * sz + g * xc * xq # Jaynes-Cummings Hamiltonian
        energies, eigenvectors = H_spectral.eigenstates()
        ground_state_energy = energies[0]
        spectral_lines[:, flux_id] = np.sort(energies[1:3]-ground_state_energy)/(2*np.pi) # 0->1 and 1->2 transitions only
        wq_flux_dep[flux_id] = wq_flux
def f(fl, q1,q2,q3):
    print(fl)
    
    
    b.set_data(fluxes+fl, spectral_lines[0])
    c.set_data(fluxes+fl, spectral_lines[1])
interactive(f, fl=(-10,-1, 0.001),
            q1=(-0.01, 0.01, 0.00001),
            q2=(-3, 3, 0.00001),
            q3=(-0.1, 0.1, 0.00001))

interactive(children=(FloatSlider(value=-6.0, description='fl', max=-1.0, min=-10.0, step=0.001), FloatSlider(value=-9.999999999999593e-06, description='q1', max=0.01, min=-0.01, step=1e-05), FloatSlider(value=4.440892098500626e-16, description='q2', max=3.0, min=-3.0, step=1e-05), FloatSlider(value=0.0, description='q3', max=0.1, min=-0.1, step=1e-05), Output()), _dom_classes=('widget-interact',))

In [ ]:
import matplotlib
%matplotlib inline
matplotlib.rcParams.update({'font.size': 18})
S21 = 10**(tt[1][3]/10)*np.exp(1j*tt[1][4])
delta = np.abs(S21.T-np.mean(S21.T, axis=0))
figs, axes = plt.subplots(1,2, figsize=(18,6))
axes[0].ticklabel_format(axis='x', style='sci', scilimits=(-2,2))
axes[0].pcolormesh(tt[1][0]*1e6, tt[1][1]/1e9, delta, vmin=0, vmax=5e-3)
axes[0].set_xlabel('Ток соленоида, мкА')
axes[0].set_ylabel('Частота, ГГц')
axes[1].pcolormesh(tt[1][0]*0.62e4+0.105, tt[1][1]/1e9, delta, vmin=0, vmax=5e-3)
plt.colorbar(axes[1].get_children()[0])
ax_lims = axes[1].get_xlim()
axes[1].plot(fluxes, spectral_lines.T, linewidth=2)
axes[1].set_xlabel('Внешний поток $\\Phi/\\Phi_0$')
axes[1].set_ylim(axes[0].get_ylim())
axes[1].set_xlim(ax_lims)